In [ ]:
!pip install wfdb
!pip install hrv-analysis
!pip install py-ecg-detectors
!pip install neurokit2

from google.colab import output
output.clear()
print("Packages installed successfully!")

Packages installed successfully!


In [ ]:
import pprint
import os
import datetime

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import wfdb
import hrvanalysis
import ecgdetectors
import hrv
import neurokit2 as nk

In [ ]:
# DOWNLOAD_PATH = "https://physionet.org/content/nsr2db/1.0.0/"
# SAVE_DIR = "/content/drive/MyDrive/Projects/HRV/dataset"
# !wget -r -N -c -np $DOWNLOAD_PATH -P $SAVE_DIR

# !wget -r -N -c -np https://physionet.org/files/nsr2db/1.0.0/ -P /content/dataset

In [ ]:
# NSR2DB_COLAB_DIR = ...
NSR2DB_GDRIVE_DIR = "/content/drive/MyDrive/Projects/HRV/dataset/physionet.org/files/"
# !cp -R /content/dataset/physionet.org/files/* $NSR2DB_GDRIVE_DIR

## Normal Sinus Rhythm RR Interval Database

- https://physionet.org/content/nsr2db/1.0.0/

In [ ]:
rec = wfdb.get_record_list('nsr2db')
print(rec)

['nsr001', 'nsr002', 'nsr003', 'nsr004', 'nsr005', 'nsr006', 'nsr007', 'nsr008', 'nsr009', 'nsr010', 'nsr011', 'nsr012', 'nsr013', 'nsr014', 'nsr015', 'nsr016', 'nsr017', 'nsr018', 'nsr019', 'nsr020', 'nsr021', 'nsr022', 'nsr023', 'nsr024', 'nsr025', 'nsr026', 'nsr027', 'nsr028', 'nsr029', 'nsr030', 'nsr031', 'nsr032', 'nsr033', 'nsr034', 'nsr035', 'nsr036', 'nsr037', 'nsr038', 'nsr039', 'nsr040', 'nsr041', 'nsr042', 'nsr043', 'nsr044', 'nsr045', 'nsr046', 'nsr047', 'nsr048', 'nsr049', 'nsr050', 'nsr051', 'nsr052', 'nsr053', 'nsr054']


In [ ]:
# NSR2DB_DIR = "/content/dataset/physionet.org/files/nsr2db/1.0.0/"
NSR2DB_DIR = "/content/drive/MyDrive/Projects/HRV/dataset/physionet.org/files/nsr2db/1.0.0/"

In [ ]:
FS = 128
h = wfdb.rdheader(os.path.join(NSR2DB_DIR, "nsr002"))
h.__dict__

{'record_name': 'nsr002',
 'n_sig': 0,
 'fs': 128,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 0,
 'base_time': None,
 'base_date': None,
 'comments': ['Age: 67 Sex: M'],
 'sig_name': None,
 'p_signal': None,
 'd_signal': None,
 'e_p_signal': None,
 'e_d_signal': None,
 'file_name': None,
 'fmt': None,
 'samps_per_frame': None,
 'skew': None,
 'byte_offset': None,
 'adc_gain': None,
 'baseline': None,
 'units': None,
 'adc_res': None,
 'adc_zero': None,
 'init_value': None,
 'checksum': None,
 'block_size': None}

In [ ]:
ann = wfdb.rdann(os.path.join(NSR2DB_DIR, "nsr002"), extension='ecg')
print(ann.__dict__.keys())

dict_keys(['record_name', 'extension', 'sample', 'symbol', 'subtype', 'chan', 'num', 'aux_note', 'fs', 'label_store', 'description', 'custom_labels', 'contained_labels', 'ann_len'])


In [ ]:
print("Symbol length    :", len(ann.symbol))
print("R-peaks length   :", len(ann.sample))

Symbol length    : 111384
R-peaks length   : 111384


In [ ]:
print(np.unique(ann.symbol))
type(ann.symbol[0])

['A' 'N' 'V' '~']


str

In [ ]:
print("Symbol samples    :", ann.symbol[:10])
print("R-peaks samples   :", ann.sample[:10])

Symbol samples    : ['~', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
R-peaks samples   : [28824 57649 57740 57834 57927 58021 58115 58209 58303 58394]


In [ ]:
__start = 64000
__end = 70000
__sam = np.array(ann.sample, dtype=np.float32)
__sym = np.array(ann.symbol)

__cond = (__start <= __sam) & (__sam < __end)
__samm = __sam[__cond]
__symm = __sym[__cond]

print(__symm, len(__symm))
print(__samm, len(__samm))

for idx, annot in enumerate(__symm):
    if str(annot).upper() != 'N':
        __samm[idx] = np.nan

xy_list = []
for x, y in zip(__samm, __symm):
    xy_list.append((x, y))
print(xy_list, len(xy_list))

['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' '~' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'V'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'] 65
[64015. 64102. 64192. 64280. 64371. 64459. 64546. 64634. 64723. 64809.
 64895. 64924. 64977. 65063. 65149. 65237. 65324. 65411. 65495. 65580.
 65664. 65749. 65835. 65920. 66008. 66096. 66181. 66268. 66355. 66443.
 66532. 66622. 66714. 66808. 66900. 66994. 67086. 67180. 67275. 67371.
 67470. 67569. 67671. 67774. 67880. 67981. 68084. 68188. 68288. 68389.
 68486. 68583. 68678. 68733. 68872. 68968. 69065. 69160. 69261. 69363.
 69470. 69577. 69684. 69794. 69903.] 65
[(64015.0, 'N'), (64102.0, 'N'), (64192.0, 'N'), (64280.0, 'N'), (64371.0, 'N'), (64459.0, 'N'), (64546.0, 'N'), (64634.0, 'N'), (64723.0, 'N'), (64809.0, 'N'), (64895.0, 'N'), (nan, '~'), (64977.0, 'N'), (65063.0, 'N'), (65149.0, 'N'), (65237.0, 'N'), (65324.0, 'N'

In [ ]:
from google.colab import output

DATABASE = "nsr2db"

start_time = datetime.datetime.now()

FS = 128

MENIT = 5
SIZE_SEC = int(60 * MENIT)
WINDOW_SIZE = SIZE_SEC * FS

END_SEC = int(60 * 120) * FS
# END_SEC = max(r_peaks)

# OVERLAP means percentage of overlap, values between [0, 1)
OVERLAP = 0.5

nsr2db = {}
for record in wfdb.get_record_list(DATABASE)[:]:
    ann = wfdb.rdann(os.path.join(NSR2DB_DIR, record), extension='ecg')
    record_hea = wfdb.rdheader(os.path.join(NSR2DB_DIR, record))

    r_peaks = np.array(ann.sample, dtype=np.float32)
    annotation = np.array(ann.symbol)

    start_sec = 0
    while start_sec < END_SEC:
        output.clear()
        end_sec = start_sec + WINDOW_SIZE

        print("New Record           :", record)
        print("  Start Sec          : %d" % int(start_sec//FS))
        print("  End Sec            : %d" % int(end_sec//FS))

        # dropping/filtering criteria
        # 1 - only take normal heartbeat
        #       & drop RRI value > 2000ms (Ludi Wang, 2019)
        # 2 - less than 10% of ectopic beats still permissible which then to be replaced with linear interpolation
        #       & drop RRI value > 3000ms
        #       & drop record whose inside first 120 minutes contains less than 10 valid 5-min segments (Alan Jovic, 2019
        # 3 - ectopic-free / normal heartbeat (Yalcin Isler, 2018)

        cond = (start_sec <= r_peaks) & (r_peaks < end_sec)
        _r_peak = r_peaks[cond]
        _annotation = annotation[cond]

        # counting ectopic beats
        abnormal_beat_num = 0
        for _sym, _count in np.transpose(np.unique(_annotation, return_counts=True)):
            if _sym != 'N':
                abnormal_beat_num += int(_count)
            if _sym == 'N':
                normal_beat_num = int(_count)
        normal_beats_ratio = normal_beat_num / (normal_beat_num + abnormal_beat_num)

        # check & replace ectopic beat with np.nan
        # assert len(_r_peak) == len(_annotation)
        for idx, ann in enumerate(_annotation):
            if str(ann).upper() != 'N':
                _r_peak[idx] = np.nan
        # interpolate
        _r_peak = hrvanalysis.preprocessing.interpolate_nan_values(_r_peak)

        # retrieve rr_interval from distance between 2 r_peak points
        rri = [_r_peak[i+1] - _r_peak[i] for i in range(len(_r_peak) - 1)]
        # convert the unit from freq_sample to milisecond
        rri = list(np.array(rri) * 1000 / FS)
        print("  len(_r_peak)           :", len(_r_peak))
        print("  len(rri)               :", len(rri))

        id = "{0}-{1}-{2}-{3}".format(DATABASE, record, start_sec//FS, end_sec//FS)
        try:
            rri = hrvanalysis.preprocessing.remove_outliers(
                rri,
                low_rri = 300,
                high_rri = 2000,
            )
            nni = hrvanalysis.preprocessing.interpolate_nan_values(rri)

            # remove nan values if exists
            nni = [val for val in nni if np.isfinite(val)]
            print("  len(nni)               :", len(nni))
            time_domain = hrvanalysis.extract_features.get_time_domain_features(nni)
            print("  time_domain computed!")
            freq_domain = hrvanalysis.extract_features.get_frequency_domain_features(nni)
            print("  freq_domain computed!")
            geom = hrvanalysis.extract_features.get_geometrical_features(nni)
            print("  geometrical features computed!")
            poincare = hrvanalysis.extract_features.get_poincare_plot_features(nni)
            print("  poincare features computed!")
            csi_cvi = hrvanalysis.extract_features.get_csi_cvi_features(nni)
            print("  csi cvi computed!")
            # Function computing the sample entropy of the given data.
            # Must use this function on short term recordings, from 1 minute window.
            sampen = hrvanalysis.extract_features.get_sampen(nni)
            print("  sampen computed!")

            nsr2db[id] = {
                "id": id,
                "start_secs": start_sec // FS,
                "end_secs": end_sec // FS,
                "db_source": DATABASE,
                "age": record_hea.comments[0].split()[1],
                "gender": record_hea.comments[0].split()[3],
                "fs": FS,
                "signal_length": np.nan,
                "recording_time_hours": np.nan,
                "recording_time_seconds": np.nan,
                "normal_beats": normal_beat_num,
                "abnormal_beats": abnormal_beat_num,
                "normal_beats_ratio": normal_beats_ratio,
                "rri_length": len(rri),
                "nni_length": len(nni),
                **time_domain,
                **freq_domain,
                **geom,
                **poincare,
                **csi_cvi,
                **sampen,
                "cardiac_info": "Sinus",
                "risk": 0
            }
            print("  RECORD {} SAVED!\n".format(id))
            output.clear()
        except:
            print("  \nERROR at {}\n".format(id))
        finally:
            start_sec = start_sec + int(WINDOW_SIZE * (1 - OVERLAP))

print(datetime.datetime.now() - start_time)

0:05:55.098635


In [ ]:
df_nsr2db = pd.DataFrame(nsr2db).T
df_nsr2db

,id,start_secs,end_secs,db_source,age,gender,fs,signal_length,recording_time_hours,recording_time_seconds,...,tinn,sd1,sd2,ratio_sd2_sd1,csi,cvi,Modified_csi,sampen,cardiac_info,risk
nsr2db-nsr001-0-300,nsr2db-nsr001-0-300,0,300,nsr2db,64,F,128,NaN,NaN,NaN,...,None,12.365084,39.798418,3.218613,3.218613,3.896183,512.382798,1.89712,Sinus,0
nsr2db-nsr001-150-450,nsr2db-nsr001-150-450,150,450,nsr2db,64,F,128,NaN,NaN,NaN,...,None,12.629396,67.437082,5.339692,5.339692,4.134401,1440.372934,1.017174,Sinus,0
nsr2db-nsr001-300-600,nsr2db-nsr001-300-600,300,600,nsr2db,64,F,128,NaN,NaN,NaN,...,None,13.439951,77.82827,5.790815,5.790815,4.223655,1802.756475,0.844154,Sinus,0
nsr2db-nsr001-450-750,nsr2db-nsr001-450-750,450,750,nsr2db,64,F,128,NaN,NaN,NaN,...,None,12.799581,69.890189,5.46035,5.46035,4.155732,1526.499558,0.658449,Sinus,0
nsr2db-nsr001-600-900,nsr2db-nsr001-600-900,600,900,nsr2db,64,F,128,NaN,NaN,NaN,...,None,10.357741,55.056883,5.31553,5.31553,3.960197,1170.626083,0.637105,Sinus,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nsr2db-nsr054-6450-6750,nsr2db-nsr054-6450-6750,6450,6750,nsr2db,35,M,128,NaN,NaN,NaN,...,None,16.763385,77.30872,4.611761,4.611761,4.31671,1426.117266,0.738984,Sinus,0
nsr2db-nsr054-6600-6900,nsr2db-nsr054-6600-6900,6600,6900,nsr2db,35,M,128,NaN,NaN,NaN,...,None,15.510041,73.653426,4.748758,4.748758,4.261926,1399.04912,0.831578,Sinus,0
nsr2db-nsr054-6750-7050,nsr2db-nsr054-6750-7050,6750,7050,nsr2db,35,M,128,NaN,NaN,NaN,...,None,16.805192,60.198384,3.58213,3.58213,4.209148,862.553772,0.846505,Sinus,0
nsr2db-nsr054-6900-7200,nsr2db-nsr054-6900-7200,6900,7200,nsr2db,35,M,128,NaN,NaN,NaN,...,None,17.570774,65.619515,3.734583,3.734583,4.265944,980.246104,0.825572,Sinus,0


In [ ]:
df_nsr2db.to_csv("df_nsr2db.csv", index=False)
df_nsr2db.to_excel("df_nsr2db.xlsx", index=False)

In [ ]:
# DIR = "30-sec"
# df_nsr2db.to_csv(os.path.join("/content/drive/MyDrive/Projects/HRV/short-term/dataset/".format(DIR), "df_nsr2db.csv"), index=False)
# df_nsr2db.to_excel(os.path.join("/content/drive/MyDrive/Projects/HRV/short-term/dataset/".format(DIR), "df_nsr2db.xlsx"), index=False)

# print("File saved!")

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
pd.read_csv("df_nsr2db.csv")

,id,start_secs,end_secs,db_source,age,gender,fs,signal_length,recording_time_hours,recording_time_seconds,normal_beats,abnormal_beats,normal_beats_ratio,rri_length,nni_length,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,lf_hf_ratio,lfnu,hfnu,total_power,vlf,triangular_index,tinn,sd1,sd2,ratio_sd2_sd1,csi,cvi,Modified_csi,sampen,cardiac_info,risk
0,nsr2db-nsr001-0-300,0,300,nsr2db,64.0,F,128,NaN,NaN,NaN,107,0,1.000000,106,106,697.597288,29.468707,17.403399,0,0.000000,28,26.666667,17.419297,703.1250,140.6250,0.024970,0.042243,86.165577,97.215190,79.175258,3.717926,253.966327,22.832709,11.122917,91.751160,8.248840,480.405662,203.606626,6.235294,NaN,12.365084,39.798418,3.218613,3.218613,3.896183,512.382798,1.897120,Sinus,0
1,nsr2db-nsr001-150-450,150,450,nsr2db,64.0,F,128,NaN,NaN,NaN,334,0,1.000000,333,333,672.391141,48.514233,17.833744,1,0.301205,94,28.313253,17.833806,679.6875,250.0000,0.026523,0.072152,89.719648,111.304348,76.039604,6.773420,1072.875464,45.147975,23.763534,95.961804,4.038196,1802.687658,684.664220,11.482759,NaN,12.629396,67.437082,5.339692,5.339692,4.134401,1440.372934,1.017174,Sinus,0
2,nsr2db-nsr001-300-600,300,600,nsr2db,64.0,F,128,NaN,NaN,NaN,471,1,0.997881,471,471,636.478238,55.847434,18.986730,2,0.425532,116,24.680851,18.986759,640.6250,320.3125,0.029831,0.087744,94.984183,112.941176,70.458716,8.207786,813.074274,79.036316,10.287350,91.140525,8.859475,1828.123931,936.013342,15.700000,NaN,13.439951,77.828270,5.790815,5.790815,4.223655,1802.756475,0.844154,Sinus,0
3,nsr2db-nsr001-450-750,450,750,nsr2db,64.0,F,128,NaN,NaN,NaN,474,1,0.997895,474,474,633.191588,50.241755,18.082196,6,1.268499,88,18.604651,18.082679,625.0000,320.3125,0.028558,0.079347,95.342327,112.941176,70.458716,7.412056,181.460630,28.465192,6.374825,86.440357,13.559643,738.179900,528.254078,11.285714,NaN,12.799581,69.890189,5.460350,5.460350,4.155732,1526.499558,0.658449,Sinus,0
4,nsr2db-nsr001-600-900,600,900,nsr2db,64.0,F,128,NaN,NaN,NaN,472,0,1.000000,471,471,636.262606,39.614033,14.632466,5,1.063830,69,14.680851,14.633608,632.8125,187.5000,0.022999,0.062261,94.655660,108.169014,80.842105,5.723024,135.350506,28.432955,4.760339,82.639911,17.360089,402.654369,238.870908,9.812500,NaN,10.357741,55.056883,5.315530,5.315530,3.960197,1170.626083,0.637105,Sinus,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2548,nsr2db-nsr054-6450-6750,6450,6750,nsr2db,35.0,M,128,NaN,NaN,NaN,540,4,0.992647,543,543,554.903315,55.935898,23.685127,18,3.321033,104,19.188192,23.685236,554.6875,390.6250,0.042684,0.100803,109.181677,137.142857,72.452830,10.627727,889.458726,275.546222,3.227984,76.348064,23.651936,1967.559877,802.554929,11.081633,NaN,16.763385,77.308720,4.611761,4.611761,4.316710,1426.117266,0.738984,Sinus,0
2549,nsr2db-nsr054-6600-6900,6600,6900,nsr2db,35.0,M,128,NaN,NaN,NaN,540,4,0.992647,543,543,554.788214,53.223061,21.914266,14,2.583026,104,19.188192,21.914569,554.6875,429.6875,0.039501,0.095934,109.124189,137.142857,69.189189,10.316240,635.054993,259.274830,2.449351,71.009037,28.990963,1587.203771,692.873949,12.928571,NaN,15.510041,73.653426,4.748758,4.748758,4.261926,1399.049120,0.831578,Sinus,0
2550,nsr2db-nsr054-6750-7050,6750,7050,nsr2db,35.0,M,128,NaN,NaN,NaN,511,0,1.000000,510,510,587.362132,44.194230,23.742773,17,3.339882,122,23.968566,23.747540,578.1250,367.1875,0.040431,0.075242,102.682163,120.000000,69.189189,7.121643,868.025390,291.349975,2.979322,74.870091,25.129909,1624.169502,464.794138,10.000000,NaN,16.805192,60.198384,3.582130,3.582130,4.209148,862.553772,0.846505,Sinus,0
2551,nsr2db-nsr054-6900-7200,6900,7200,nsr2db,35.0,M,128,NaN,NaN,NaN,490,0,1.000000,489,489,613.097520,48.034638,24.823355,19,3.893443,123,25.204918,24.823979,609.3750,281.2500,0.040489,0.078347,98.424059,112.941176,73.846154,7.206889,978

In [ ]:
pd.read_excel("df_nsr2db.xlsx")

,id,start_secs,end_secs,db_source,age,gender,fs,signal_length,recording_time_hours,recording_time_seconds,normal_beats,abnormal_beats,normal_to_abnormal_ratio_beats,rri_length,nni_length,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,lf_hf_ratio,lfnu,hfnu,total_power,vlf,triangular_index,tinn,sd1,sd2,ratio_sd2_sd1,csi,cvi,Modified_csi,sampen,cardiac_info,risk
0,nsr2db-nsr001-0-300,0,300,nsr2db,64.0,F,128,NaN,NaN,NaN,107,0,1.000000,106,106,697.597288,29.468707,17.403399,0,0.000000,28,26.666667,17.419297,703.1250,140.6250,0.024970,0.042243,86.165577,97.215190,79.175258,3.717926,253.966327,22.832709,11.122917,91.751160,8.248840,480.405662,203.606626,6.235294,NaN,12.365084,39.798418,3.218613,3.218613,3.896183,512.382798,1.897120,Sinus,0
1,nsr2db-nsr001-150-450,150,450,nsr2db,64.0,F,128,NaN,NaN,NaN,334,0,1.000000,333,333,672.391141,48.514233,17.833744,1,0.301205,94,28.313253,17.833806,679.6875,250.0000,0.026523,0.072152,89.719648,111.304348,76.039604,6.773420,1072.875464,45.147975,23.763534,95.961804,4.038196,1802.687658,684.664220,11.482759,NaN,12.629396,67.437082,5.339692,5.339692,4.134401,1440.372934,1.017174,Sinus,0
2,nsr2db-nsr001-300-600,300,600,nsr2db,64.0,F,128,NaN,NaN,NaN,471,1,0.997881,471,471,636.478238,55.847434,18.986730,2,0.425532,116,24.680851,18.986759,640.6250,320.3125,0.029831,0.087744,94.984183,112.941176,70.458716,8.207786,813.074274,79.036316,10.287350,91.140525,8.859475,1828.123931,936.013342,15.700000,NaN,13.439951,77.828270,5.790815,5.790815,4.223655,1802.756475,0.844154,Sinus,0
3,nsr2db-nsr001-450-750,450,750,nsr2db,64.0,F,128,NaN,NaN,NaN,474,1,0.997895,474,474,633.191588,50.241755,18.082196,6,1.268499,88,18.604651,18.082679,625.0000,320.3125,0.028558,0.079347,95.342327,112.941176,70.458716,7.412056,181.460630,28.465192,6.374825,86.440357,13.559643,738.179900,528.254078,11.285714,NaN,12.799581,69.890189,5.460350,5.460350,4.155732,1526.499558,0.658449,Sinus,0
4,nsr2db-nsr001-600-900,600,900,nsr2db,64.0,F,128,NaN,NaN,NaN,472,0,1.000000,471,471,636.262606,39.614033,14.632466,5,1.063830,69,14.680851,14.633608,632.8125,187.5000,0.022999,0.062261,94.655660,108.169014,80.842105,5.723024,135.350506,28.432955,4.760339,82.639911,17.360089,402.654369,238.870908,9.812500,NaN,10.357741,55.056883,5.315530,5.315530,3.960197,1170.626083,0.637105,Sinus,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2548,nsr2db-nsr054-6450-6750,6450,6750,nsr2db,35.0,M,128,NaN,NaN,NaN,540,4,0.992647,543,543,554.903315,55.935898,23.685127,18,3.321033,104,19.188192,23.685236,554.6875,390.6250,0.042684,0.100803,109.181677,137.142857,72.452830,10.627727,889.458726,275.546222,3.227984,76.348064,23.651936,1967.559877,802.554929,11.081633,NaN,16.763385,77.308720,4.611761,4.611761,4.316710,1426.117266,0.738984,Sinus,0
2549,nsr2db-nsr054-6600-6900,6600,6900,nsr2db,35.0,M,128,NaN,NaN,NaN,540,4,0.992647,543,543,554.788214,53.223061,21.914266,14,2.583026,104,19.188192,21.914569,554.6875,429.6875,0.039501,0.095934,109.124189,137.142857,69.189189,10.316240,635.054993,259.274830,2.449351,71.009037,28.990963,1587.203771,692.873949,12.928571,NaN,15.510041,73.653426,4.748758,4.748758,4.261926,1399.049120,0.831578,Sinus,0
2550,nsr2db-nsr054-6750-7050,6750,7050,nsr2db,35.0,M,128,NaN,NaN,NaN,511,0,1.000000,510,510,587.362132,44.194230,23.742773,17,3.339882,122,23.968566,23.747540,578.1250,367.1875,0.040431,0.075242,102.682163,120.000000,69.189189,7.121643,868.025390,291.349975,2.979322,74.870091,25.129909,1624.169502,464.794138,10.000000,NaN,16.805192,60.198384,3.582130,3.582130,4.209148,862.553772,0.846505,Sinus,0
2551,nsr2db-nsr054-6900-7200,6900,7200,nsr2db,35.0,M,128,NaN,NaN,NaN,490,0,1.000000,489,489,613.097520,48.034638,24.823355,19,3.893443,123,25.204918,24.823979,609.3750,281.2500,0.040489,0.078347,98.424059,112.941176,73.846154,